In [ ]:
pip install pillow

In [ ]:
pip install numpy

In [ ]:
pip install scikit-image

In [ ]:
pip install tensorflow

In [ ]:
import os,sys,glob,time
import numpy as np
from PIL import Image, ImageDraw
from skimage.io import imshow, imread
import tensorflow as tf
path = os.getcwd()
print(path)

In [ ]:
## Загрузка сетей в память
Net =[]
t = time.time()
Net.append(tf.keras.models.load_model(path + "/net/Зд.ул.h5")) ## Неправильная сеть
Net.append(tf.keras.models.load_model(path + "/net/Зд.ул.h5"))
Net.append(tf.keras.models.load_model(path + "/net/Д.н.h5"))
Net.append(tf.keras.models.load_model(path + "/net/Л.н.h5"))
Net.append(tf.keras.models.load_model(path + "/net/Од.мн.h5"))
print("%s сек. Понадобилось для загрузки сетей" % round(time.time()-t,2))

In [ ]:
# Функция сосздания белого холста
def im_in_white(im):
    draw = ImageDraw.Draw(im)
    width = im.size[0]
    height = im.size[1]
    pix = im.load()
    for i in range(width):
        for j in range(height):
            draw.point((i,j),(255,255,255))
    return(im)

# Подгон изображения в рамки размера
def podgon(path,size):
    imge = Image.open(path)
    imshow(imread(path))
    shape = imge.size
    if shape[0]>=shape[1]: 
        k = size[0]/shape[0]
    else:
         k = size[1]/shape[1]
    height = round(shape[0]*k)
    widht  = round(shape[1]*k)
    out_imge = imge.resize((height , widht))
    return(out_imge)

# Вставка изображения в холст
def past_im(holst,path_im,im):
    if im == None:  im = Image.open(path_im)
    draw = ImageDraw.Draw(holst)
    width = im.size[0]
    height = im.size[1]
    pix = im.load()
    for i in range(width):
        for j in range(height):
            r = pix[i,j][0] 
            g = pix[i,j][1] 
            b = pix[i,j][2]
            draw.point((i,j),(r,g,b))
    return(holst)


# Интерпритация выхода сети в вероятность
def predict_(inp):
    if inp[0]>= inp[1]:
        v = round(inp[0]*100)
        return [1,v]
    else:
        v = round(inp[1]*100)
        return[0,v];  

# Функция объединяющая предсказания сетей в удобну форму для восприятия    
size = (400,400,3)    
def selecting_photos(n):
    t = time.time()
    path_ = path +"/test/1 ({}).jpg".format(n);
    predict = []
    holst = im_in_white(Image.new("RGB",size[:2]))
    imge = past_im(holst,'',im=podgon(path_,size[:2]))
    tabe = np.array(imge)/255
    tabe = tabe.reshape(1,400,400,3)
    print("————————————————————————————————————————————————————>")
    predict.append(predict_([0,100])) ## Предсказание ещё не обученной сети
    if predict[0][0] == 1: 
        print("На фотографии текст с вероятностью {}_%".format(predict[0][1]))
    else:
        predict.append(predict_(Net[1].predict(tabe)[0]))
        if predict[1][0] == 1: 
            print("Фотография сделана в здании с вероятностью {}_%".format(predict[1][1]))
        else:    
            print("Фотография сделана на улице с вероятностью {}_%".format(predict[1][1]))
        predict.append(predict_(Net[2].predict(tabe)[0]))
        if predict[2][0] == 1: 
            print("Фотография сделана днём с вероятностью {}_%".format(predict[2][1]))
        else:    
            print("Фотография сделана ночью с вероятностью {}_%".format(predict[2][1]))
        predict.append(predict_(Net[3].predict(tabe)[0]))
        if predict[3][0] == 1: 
            print("На фотографии присутствуют люди с вероятностью {}_%".format(predict[3][1]))
            predict.append(predict_(Net[4].predict(tabe)[0]))
            if predict[4][0] == 0: 
                    print("На фотографии более одного человека с вероятностью {}_%".format(predict[4][1]))
            else:    
                    print("На фотографии один человек с вероятностью {}_%".format(predict[4][1]))
        else:    
            print("На фотографии нет людей с вероятностью {}_%".format(predict[3][1]))
            predict.append(predict_([0,0]))
    t = round(time.time() - t,2)
    print("Сеть дала ответ за %s сек." % t)
    print("<————————————————————————————————————————————————————")
    holst = im_in_white(holst)
    

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# Исполнительный блок
n = int(input("Номер теста от 1 до 30 ———>>> "))
if (n>0) and (n<=30):
    selecting_photos(n)
else:
    print("Такого теста нет")
    

In [ ]:
## Прогон сети по N тестам
t=time.time()
N = 40
for i in range(N):
    selecting_photos(i+1);
print(time.time()-t)